In [ ]:
TEXT =  "This is a cat ."
# Set to an English sentence to translate; if None, reads from stdin or input()

In [ ]:
# -------------------------------
# User-configurable variables
# -------------------------------
CPU = False  # Set True to force CPU inference

# Model hyperparameters (must match training)
HIDDEN_SIZE = 333
NHEADS = 9
ENC_LAYERS = 4
DEC_LAYERS = 4
DROPOUT = 0.15
DIM_FF = -1  # If <= 0, will default to 4 * HIDDEN_SIZE

# Decoding parameters
BEAM_WIDTH = 8
MAX_LEN = 150
LENGTH_PENALTY = 1
NO_REPEAT_NGRAM = 3
REPETITION_PENALTY = 2
MIN_LENGTH = 5
TEMPERATURE = 3
# Languages to translate into
LANGS = ["Bengali", "Hindi"]
# -------------------------------

# Reserved tokens
RESERVED_TOKENS = ["<pad>", "<unk>", "<s>", "</s>"]
WHITESPACE_MARKER = "▁"

In [ ]:
import os
import sys
import math
import torch
import torch.nn as nn
from huggingface_hub import hf_hub_download, HfApi  # ensures hub deps available via utils.prepare
from worker import TransformerEncoder, TransformerDecoder  # model definitions
from utils import (
    prepare,
    detokenize_sentence,
    encode_and_pad,
    beam_search_translate_transformer,
    _decode_token_text,
)
import re
import unicodedata
import warnings
warnings.filterwarnings("ignore")
BASE = "/kaggle/working/" if "KAGGLE_KERNEL_RUN_TYPE" in os.environ else ""
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

In [ ]:
def normalize_text(text: str) -> str:
    # Unicode NFKC normalization and lowercasing to match training flow
    normalized = unicodedata.normalize("NFKC", text)
    return normalized.casefold()

def tokenize_sentence(text: str, tokenizer_data):
    # Greedy longest-match tokenization over prepared vocab
    vocab, _ = tokenizer_data
    vocab_map = {_decode_token_text(tok): i for i, tok in enumerate(vocab)}
    text = normalize_text(text)
    processed_text = WHITESPACE_MARKER + re.sub(r"\s+", WHITESPACE_MARKER, text.strip())
    ids = []
    i = 0
    while i < len(processed_text):
        longest_match = ""
        for j in range(len(processed_text), i, -1):
            sub = processed_text[i:j]
            if sub in vocab_map:
                longest_match = sub
                break
        if longest_match:
            ids.append(vocab_map[longest_match])
            i += len(longest_match)
        else:
            ids.append(vocab_map["<unk>"])
            i += 1
    return ids

def load_models_and_tokenizers(lang: str, device):
    # Load prepared tokenizers bundle from HF using utils.prepare
    # Prepared file contains: en_tokenizer_data, target_tokenizer_data
    _, en_tok_data, tgt_tok_data, _, _ = prepare(lang)
    en_map = {_decode_token_text(tok): i for i, tok in enumerate(en_tok_data[0])}
    tgt_map = {_decode_token_text(tok): i for i, tok in enumerate(tgt_tok_data[0])}
    en_vocab_size = len(en_tok_data[0])
    tgt_vocab_size = len(tgt_tok_data[0])
    SOS_ID = tgt_map["<s>"]
    EOS_ID = tgt_map["</s>"]
    PAD_ID_SRC = en_map["<pad>"]
    PAD_ID_TGT = tgt_map["<pad>"]
    # Model hyperparameters should match training
    d_model = HIDDEN_SIZE
    nheads = NHEADS
    num_layers_enc = ENC_LAYERS
    num_layers_dec = DEC_LAYERS
    dim_ff = DIM_FF if DIM_FF and DIM_FF > 0 else 4 * d_model
    dropout = DROPOUT
    encoder = TransformerEncoder(
        en_vocab_size, d_model, nheads, num_layers_enc, dim_ff, dropout, PAD_ID_SRC
    ).to(device)
    decoder = TransformerDecoder(
        d_model, tgt_vocab_size, nheads, num_layers_dec, dim_ff, dropout, PAD_ID_TGT
    ).to(device)
    # Resolve weight paths
    user_secrets = UserSecretsClient()
    enc_path = os.path.join(BASE, f"encoder_{lang}.pt")
    dec_path = os.path.join(BASE, f"decoder_{lang}.pt")
    if not os.path.exists(enc_path) or not os.path.exists(dec_path):
        enc_path = hf_hub_download(
            repo_id="Arnab-Datta-240185/CS779-Capstone-Project",
            filename=f"encoder_{lang}.pt",
            repo_type="dataset",
            token=user_secrets.get_secret("hf_token")
        )
        dec_path = hf_hub_download(
            repo_id="Arnab-Datta-240185/CS779-Capstone-Project",
            filename=f"decoder_{lang}.pt",
            repo_type="dataset",
            token=user_secrets.get_secret("hf_token")
        )
    encoder.load_state_dict(torch.load(enc_path, map_location=device))
    decoder.load_state_dict(torch.load(dec_path, map_location=device))
    encoder.eval()
    decoder.eval()
    return {
        "encoder": encoder,
        "decoder": decoder,
        "en_tok_data": en_tok_data,
        "tgt_tok_data": tgt_tok_data,
        "en_map": en_map,
        "SOS_ID": SOS_ID,
        "EOS_ID": EOS_ID,
        "PAD_ID_SRC": PAD_ID_SRC,
    }

def translate_text(text: str, lang: str):
    device = torch.device("cuda" if torch.cuda.is_available() and not CPU else "cpu")
    torch.set_float32_matmul_precision("high")
    bundle = load_models_and_tokenizers(lang, device)

    # Tokenize and encode source
    src_ids = tokenize_sentence(text, bundle["en_tok_data"])
    src_padded = encode_and_pad(bundle["en_map"], src_ids, MAX_LEN)
    src_tensor = torch.tensor(src_padded, dtype=torch.long, device=device).unsqueeze(0)

    # Decode via beam search
    out_ids = beam_search_translate_transformer(
        bundle["encoder"],
        bundle["decoder"],
        src_tensor,
        bundle["SOS_ID"],
        bundle["EOS_ID"],
        src_pad_id=bundle["PAD_ID_SRC"],
        beam_width=BEAM_WIDTH,
        max_length=MAX_LEN,
        length_penalty=LENGTH_PENALTY,
        no_repeat_ngram_size=NO_REPEAT_NGRAM,
        repetition_penalty=REPETITION_PENALTY,
        min_length=MIN_LENGTH,
        temperature=TEMPERATURE,
    )
    return detokenize_sentence(out_ids, bundle["tgt_tok_data"])


if __name__ == "__main__":
    try:
        text = TEXT

        for lang in LANGS:
            translation = translate_text(text, lang)
            print(translation)
    except Exception as e:
        print(f"Translation failed: {e}", file=sys.stderr)
        sys.exit(2)